In [26]:
#% %run  ../coronavirus.py
from coronavirus import *

In [6]:
d, c = fetch_deaths(), fetch_cases()

Downloaded data: last data point 4/6/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


In [12]:
countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

In [22]:
def modify_template(templatefile, outputfile, mappings):
    """Create concrete *.ipynb file from template"""
    # open template
    with open(templatefile, "tr") as f_template:
        template = f_template.read()
    for key in mappings:
        template = template.replace(key, mappings[key])
    with open(outputfile, "tw") as f:
        f.write(template)


In [54]:
def check_country_name_is_known(name):
    d = fetch_deaths()
    assert name in d.index, f"{name} is unknown. Known countries are {sorted(d.index)}"

def sanitize(name):
    """Given a country name as a string, sanitise it for use as URL and filename: 
    - get rid of spaces, commas
    
    return cleaned string.
    
    (Leave umlaute for now)
    """
    s = name.replace(" ", "-")
    s = s.replace(",", "-")
    return s
    

def create_ipynb_for_country(country):
    check_country_name_is_known(country)
    
    country = sanitize(country)
    mappings = {
        "%title%" : country,
        "%title2%" : "(data from Johns Hopkins)",
        "%country%" : country
    }
    modify_template("template-country.ipynb", "html/{country}.ipynb", mappings)


In [64]:
import subprocess
def nbconvert_ipynb2html(ipynb_path):
    command = f"jupyter-nbconvert {ipynb_path} --to html--execute"
    print(f"Command = {command}")
    output = subprocess.check_output(command, shell=True)
    print(f"Output is {output.decode()}")

In [65]:
country = "Italy"
create_ipynb_for_country(country)
nbconvert_ipynb2html(os.path.join('html', country + '.ipynb'))    

Command = jupyter-nbconvert html/Italy.ipynb --to html  --execute


CalledProcessError: Command 'jupyter-nbconvert html/Italy.ipynb --to html  --execute' returned non-zero exit status 255.

In [62]:
nbconvert("html/test.ipynb")


Command = jupyter-nbconvert html/test.ipynb --to html  --execute
Output is 
